For this entire project i have used only titles of a you tube videos in a channel. because the you tybe transcript api did not worked.

In [ ]:
pip install youtube-transcript-api==0.6.2

In [ ]:
import requests # api calls
import json # CONVERT TO JSON
import polars as pl # faster version of pandas
from google.colab import userdata # load secreats

from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
import os
my_key = userdata.get('my_key')

In [ ]:
def getVideoRecords(response: requests.models.Response) -> list:
    """
        Function to extract YouTube video data from GET request response
    """

    video_record_list = []

    for raw_item in json.loads(response.text)['items']:

        # only execute for youtube videos
        if raw_item['id']['kind'] != "youtube#video":
            continue

        video_record = {}
        video_record['video_id'] = raw_item['id']['videoId']
        video_record['datetime'] = raw_item['snippet']['publishedAt']
        video_record['title'] = raw_item['snippet']['title']
        #video_record['transcript'] = raw_item['snippet']['transcript']
        #video_record['description'] = raw_item['snippet']['description']

        video_record_list.append(video_record)

    return video_record_list

In [ ]:
# define channel ID
channel_id = 'UCa9gErQ9AE5jT2DZLjXBIdA'
# https://www.youtube.com/@SuprOrdinary  recomended by chat GPT
# define url for API

url = 'https://www.googleapis.com/youtube/v3/search'

# initialize page token
page_token = None

# intialize list to store video data
video_record_list = []

In [ ]:
%%time
# extract video data across multiple search result pages
while page_token != 0:
    # define parameters for API call
    # max result is reduced to 2. because it exceed the daily limit.
    params = {"key": my_key, 'channelId': channel_id, 'part': ["snippet","id"], 'order': "date", 'maxResults':2, 'pageToken': page_token}
    # make get request
    response = requests.get(url, params=params)

    # Check if the request was successful and the response contains 'items'
    if response.status_code == 200 and 'items' in json.loads(response.text):
        # append video records to list
        video_record_list += getVideoRecords(response)

        try:
            # grab next page token
            page_token = json.loads(response.text)['nextPageToken']
        except KeyError:
            # if no next page token kill while loop
            page_token = 0
    else:
        # If the request was not successful or 'items' is missing, stop the loop
        print(f"API request failed with status code {response.status_code} or no items found.")
        print(response.text) # Print the response text for debugging
        page_token = 0

CPU times: user 393 ms, sys: 34.1 ms, total: 427 ms
Wall time: 34.5 s


In [ ]:
# write data to file
os.makedirs('data')
pl.DataFrame(video_record_list).write_parquet('data/video-ids.parquet')
pl.DataFrame(video_record_list).write_csv('data/video-ids.csv')

In [ ]:
video_record_list

[{'video_id': 'QxLXhE1fxc4',
  'datetime': '2025-08-31T08:00:23Z',
  'title': 'uv: The Fastest Way to Install (and Use) Python'},
 {'video_id': 'enBm0jLXLZ4',
  'datetime': '2025-08-24T08:00:44Z',
  'title': 'GitHub for AI Engineers (beginner-friendly guide)'},
 {'video_id': 'zKHSpwayPBU',
  'datetime': '2025-08-17T08:00:59Z',
  'title': 'Context Engineering Explained (5 Practical Tips)'},
 {'video_id': 'hugQUr4VwRA',
  'datetime': '2025-08-10T08:01:00Z',
  'title': 'How to Solve Problems with AI (not just use it)'},
 {'video_id': 'fAFJYbtTsC0',
  'datetime': '2025-07-27T08:00:45Z',
  'title': 'Fine-tuning LLMs for Tool Use (w/ Example Code)'},
 {'video_id': 'uZvR1tG4Dng',
  'datetime': '2025-07-20T15:01:27Z',
  'title': 'How I Found My Ideal Customer with AI (ICP Analysis)'},
 {'video_id': 'bNmC_N4X1lk',
  'datetime': '2025-07-13T15:01:02Z',
  'title': 'How to Solve Problems with Text Embeddings and ML (Dallas AI - Summer School 2025)'},
 {'video_id': 'U9XDqQmV1Hc',
  'datetime': '202

# Get the Titles

In [ ]:
import polars as pl

In [ ]:
def extract_text(transcript: list) -> str:
    """
        Function to extract text from transcript dictionary
    """

    text_list = [transcript[i]['text'] for i in range(len(transcript))]
    return ' '.join(text_list)

In [ ]:
df = pl.read_parquet('data/video-ids.parquet')
print(df.head())

shape: (5, 3)
┌─────────────┬──────────────────────┬─────────────────────────────────┐
│ video_id    ┆ datetime             ┆ title                           │
│ ---         ┆ ---                  ┆ ---                             │
│ str         ┆ str                  ┆ str                             │
╞═════════════╪══════════════════════╪═════════════════════════════════╡
│ QxLXhE1fxc4 ┆ 2025-08-31T08:00:23Z ┆ uv: The Fastest Way to Install… │
│ enBm0jLXLZ4 ┆ 2025-08-24T08:00:44Z ┆ GitHub for AI Engineers (begin… │
│ zKHSpwayPBU ┆ 2025-08-17T08:00:59Z ┆ Context Engineering Explained … │
│ hugQUr4VwRA ┆ 2025-08-10T08:01:00Z ┆ How to Solve Problems with AI … │
│ fAFJYbtTsC0 ┆ 2025-07-27T08:00:45Z ┆ Fine-tuning LLMs for Tool Use … │
└─────────────┴──────────────────────┴─────────────────────────────────┘


In [ ]:
import pandas as pd
pd.read_csv("data/video-ids.csv")

,video_id,datetime,title
0,QxLXhE1fxc4,2025-08-31T08:00:23Z,uv: The Fastest Way to Install (and Use) Python
1,enBm0jLXLZ4,2025-08-24T08:00:44Z,GitHub for AI Engineers (beginner-friendly guide)
2,zKHSpwayPBU,2025-08-17T08:00:59Z,Context Engineering Explained (5 Practical Tips)
3,hugQUr4VwRA,2025-08-10T08:01:00Z,How to Solve Problems with AI (not just use it)
4,fAFJYbtTsC0,2025-07-27T08:00:45Z,Fine-tuning LLMs for Tool Use (w/ Example Code)
...,...,...,...
146,MX7ymkYGiZ0,2020-12-21T00:24:45Z,The Wavelet Transform | Introduction &amp; Exa...
147,rPUytg38b6Q,2020-12-04T01:10:36Z,The Fast Fourier Transform | How does it (actu...
148,mj86XmfOniY,2020-11-15T20:41:53Z,"Time Series, Signals, &amp; the Fourier Transf..."
149,Gwz4zXPeP_Q,2020-11-12T22:58:00Z,biometricDahboard3 DEMO
